In [ ]:

!pip install --upgrade --no-cache-dir gdown
!pip install rembg[gpu] gradio==3.45.0

In [ ]:

! cd openpose/build && make -j`nproc`
! cd openpose && mkdir output

In [ ]:
# downloading LIP dataset model
!gdown --id 1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
!mv /content/Self-Correction-Human-Parsing/exp-schp-201908261155-lip.pth /content/Self-Correction-Human-Parsing/checkpoints/final.pth
     

In [ ]:

%cd /content/

In [ ]:

%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/
MINICONDA_INSTALLER_SCRIPT
./
MINICONDA_PREFIX
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

In [ ]:

import sys
_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))

In [ ]:


!conda install --channel conda-forge featuretools --yes

In [ ]:
def make_dir():
  os.system("cd /content/ && mkdir inputs && cd inputs && mkdir test && cd test && mkdir cloth cloth-mask image image-parse openpose-img openpose-json")
     

In [ ]:


from PIL import Image

def run(cloth, model):
  make_dir()
  cloth.save("/content/inputs/test/cloth/cloth.jpg")
  model.save("/content/inputs/test/image/model.jpg")

  # running script to compute the predictions
  os.system("rm -rf /content/output/")
  os.system("python /content/clothes-virtual-try-on/run.py")

  # loading output
  op = os.listdir("/content/output")[0]
  op = Image.open(f"/content/output/{op}")
  return op
     

In [ ]:
import gradio as gr

with gr.Blocks() as app:
    with gr.Row():
        gr.Markdown(" Clothes Virtual Try ON ")
    with gr.Row():
        with gr.Column():
            cloth_input = gr.Image(sources=['upload'], type="pil", label="Upload the Cloth Image")
        with gr.Column():
            model_input = gr.Image(sources=['upload'], type="pil", label="Upload the Human Image")
    with gr.Row():
        final_output = gr.Image(sources=['upload'], type="pil", label="Final Prediction")
    with gr.Row():
        submit_button = gr.Button("Submit")

    submit_button.click(fn=run, inputs=[cloth_input, model_input],
                        outputs=[final_output])
app.launch(debug=True, share=True)
     